In [7]:
import enviroments_package
import gymnasium

world_dir = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/simulation_package/worlds/my_frst_webots_world.wbt"
json_take_off = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/configs/reward_package_config/takeoff.json"
json_basic = "/Users/jeste/Desktop/Clase/TFG/drone_tfg_juanes/configs/reward_package_config/basic_no_roll.json"

In [8]:
import gymnasium
from stable_baselines3.common.env_util import SubprocVecEnv
from enviroments_package import RemoveKeyObservationWrapper, ScaleRewardWrapper, ScaleActionWrapper

# Define el número de entornos que quieres crear
num_envs = 4  # Cambia este valor según tus necesidades

# Define la función make_env
def make_env():
    def _init():
        # Crea el entorno base
        env = gymnasium.make('drone_tfg_juanes/Drone-v1', simulation_path=world_dir, reward_json_path=json_basic, no_render=False)

        # Aplica los wrappers necesarios
        env = RemoveKeyObservationWrapper(env, remove_keys=["camera", "gps"])
        env = ScaleRewardWrapper(env, scale_factor=0.1)
        env = ScaleActionWrapper(env, in_low=0, in_high=2, out_low=0, out_high=576)
        return env
    return _init

# Usa make_vec_env para crear un DummyVecEnv con los entornos creados por make_env
env = SubprocVecEnv([make_env() for _ in range(num_envs)])


In [9]:
from stable_baselines3.common.callbacks import BaseCallback


class TrainingCallback(BaseCallback):
    def __init__(self, env, verbose=1):
        super(TrainingCallback, self).__init__(verbose)
        self.env = env


    def _on_step(self) -> bool:
        return True

    def _on_rollout_start(self) -> None:
        self.env.reset()

    def _on_training_end(self):
        # Este método se llama al final del entrenamiento
        print("Entrenamiento finalizado. Cerrando el entorno...")
        self.env.close()  # Cerrar el entorno para evitar problemas


# Usar el callback al entrenar

callback = TrainingCallback(env=env)

In [10]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.logger import configure

timesteps = 20480
log_dir = "./logs/ppo_training/"
os.makedirs(log_dir, exist_ok=True)

# Configurar el logger con salida en csv, log, stdout y tensorboard
new_logger = configure(log_dir, ["stdout", "csv", "log", "tensorboard"])

# Verificar si el archivo ppomodel existe en la carpeta ./models/
if not os.path.exists('./models/ppomodel'):
    # Crear y entrenar el modelo solo si el archivo no existe
    model = PPO(
        "MultiInputPolicy",
        env,
        verbose=1,
        n_steps=1024,        # Tamaño típico, controla el buffer de experiencias para actualizar la política
        batch_size=64,      # Tamaño del lote, normalmente 64 o 128
        learning_rate=3e-4, # Tasa de aprendizaje típica en PPO
        ent_coef=0.15       # Coeficiente de entropía para exploración, ajuste común en PPO
    )
    model.set_logger(new_logger)  # Asignar el nuevo logger al modelo
    model.learn(total_timesteps=timesteps, callback=callback)
    model.save('./models/ppomodel')  # Guardar el modelo para futuras ejecuciones
else:
    # Cargar el modelo guardado
    model = PPO.load("./models/ppomodel", env=env)
    model.set_logger(new_logger)  # Asignar el nuevo logger al modelo cargado
    # Continuar el entrenamiento desde el último punto
    model.learn(total_timesteps=timesteps, callback=callback)  # Ajusta los timesteps si quieres más entrenamiento
    model.save(path="./models/ppomodel")

Logging to ./logs/ppo_training/
Using cuda device
-----------------------------
| time/              |      |
|    fps             | 26   |
|    iterations      | 1    |
|    time_elapsed    | 157  |
|    total_timesteps | 4096 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 26          |
|    iterations           | 2           |
|    time_elapsed         | 313         |
|    total_timesteps      | 8192        |
| train/                  |             |
|    approx_kl            | 0.039230138 |
|    clip_fraction        | 0.204       |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.91       |
|    explained_variance   | 8.76e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 20.1        |
|    n_updates            | 10          |
|    policy_gradient_loss | 0.0108      |
|    std                  | 1.11        |
|    value_loss           

In [11]:
import os
import shutil
from datetime import datetime


def move_and_rename_csv(src_dir, dst_dir, new_name):
    # Buscar el archivo CSV en el directorio fuente
    csv_files = [f for f in os.listdir(src_dir) if f.endswith('.csv')]

    # Verificar si hay algún archivo CSV en el directorio de origen
    if not csv_files:
        print("No se encontró ningún archivo CSV en el directorio de origen.")
        return

    # Tomar el primer archivo CSV encontrado
    csv_file = csv_files[0]
    src_path = os.path.join(src_dir, csv_file)
    dst_path = os.path.join(dst_dir, new_name)

    # Mover y renombrar el archivo
    shutil.copy2(src_path, dst_path)
    print(f"Archivo copiado y renombrado a {dst_path}")


# Ejemplo de uso
src_directory = log_dir
dst_directory = './data_collected/'  # Cambia por la ruta de tu carpeta de destino
new_filename = f'ppo{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'  # Cambia por el nombre deseado del archivo en el destino

move_and_rename_csv(src_directory, dst_directory, new_filename)

Archivo copiado y renombrado a ./data_collected/ppo20241107_172639.csv


observation, _ = env.reset()

for i in range(100):  # Cambia el número de pasos según quieras observar
    action, _states = model.predict(observation)  # Usa deterministic=True para ver la política aprendida
    observation, reward, terminated, truncated, _ = env.step(action)

    if terminated:
        observation, _ = env.reset()  # Reiniciar el entorno al final del episodio

# Cerrar el entorno después de la prueba
env.close()